This program will simulate a random walk on the n-cycle and produce an animation showing how the proportion of time spent at each state changes over the course of the walk.

First, make sure you have the R packages "ggplot2" and "gganimate":

In [ ]:
library(ggplot2)
library(gganimate)

Create the simulation:

In [2]:
# Animation of the random walk on the n-cycle
# Produces an animated bar graph showing the proportion of time the walk has spent 
# at each state at each time step
#
# n = size of cycle (number of vertices)
# p = P(X, X+1)
# s = number of steps
ncycleanimation <- function(n, p, s) {
  
  # s by n matrix of number of total visits to each state at each step
  vertexCounts <- matrix(1:(s*n), nrow=s, ncol=n) 
  for (i in 1:s) {
    for (j in 1:n) {
      vertexCounts[i, j] <- 0 #initialize every entry to 0
    }
  }
  
  #starting state:
  #I'm considering the (randomly chosen) starting state to be "time 0" and it is not included in the count
  x <- floor(runif(1)*n + 1) #uniform random integer between 1 and n
  
  #run the walk for s steps
  step <- 0
  for (i in 1:s) {
    if (runif(1) < p) { #with probability p
      if (x != n) {
        x <- x + 1
      } else {
        x <- 1
      }
    } else {
      if (x != 1) {
        x <- x - 1
      } else {
        x <- n
      }
    }
    step <- step + 1
    if (step == 1) { #if this is the first step
      vertexCounts[step, x] <- vertexCounts[step, x] + 1 #add 1 to the count of visits to state x
    } else { #if this isn't the first step
      for (i in 1:n) { #assign the counts for the current step to be the same as the previous step
        vertexCounts[step, i] <- vertexCounts[step - 1, i] 
      }
      vertexCounts[step, x] <- vertexCounts[step - 1, x] + 1 #record the visit to state x
    }
  } 
  
  ## These next few steps create a data frame that tells us the number of visits to each state
  ## by each step of the process.
  
  # "Day" lists all the steps from 1 to s, with each repeated n times
  # For example, if n=3 and s=5, "Day" will be (1,1,1,2,2,2,3,3,3,4,4,4,5,5,5)
  Day <- c()
  for (i in 1:s) {
    Day <- c(Day, rep(i, n)) #all the numbers from 1 to s, with each number repeated n times
  }
  
  # "State" lists all the numbers from 1 through n, repeated s times
  # For example, for n=3 and s=5, "State" will be (1,2,3,1,2,3,1,2,3,1,2,3,1,2,3)
  State <- rep(c(1:n), s) 
  
  # "Visits" describes the number of visits at each state at each step
  Visits <- c()
  for (i in 1:s) {
    for (j in 1:n) {
      Visits <- c(Visits, vertexCounts[i, j]) #combines every row of vertexCounts into one long vector
    }
  }
  
  # Create the data frame, with Day, State, and Visits as the columns
  history <- data.frame("Day"=Day, "State"=State, "Visits"=Visits) 
  
  # Create a new column listing the proportions of visits to each state at each step
  history$Proportion <- Visits / Day 
  
  # Bar graph of the proportion of visits to each state at each step, using the ggplot2 package.
  # Note: you can easily change this to show the number of visits to each state at each step by
  # changing "y=Proportion" to "y=Visits" (and changing the y axis label to say "Number of Visits")
  walk <- ggplot(history, aes(x=factor(State), y=Proportion)) + geom_bar(position=position_dodge(),
            stat="identity", color="black",width=0.8) + theme_bw() +
            labs(title=paste("Random Walk on the", n, "cycle for", s, "steps, p =", p), 
                 x="State", y="Proportion of Time") + geom_hline(yintercept=1/n, linetype="dashed", color="blue")
  
  # This uses the gganimate package to show how the proportion of visits to each state
  # changes over the course of the random walk. It separates the data by Day, meaning it
  # will show the graph for step 1, then step 2, etc, all the way to step s.
  walk + transition_states(Day,
         transition_length = 1,
         state_length = 1)   
}


Let's try a walk on the 6-cycle for 50 steps with p=0.5:

In [3]:
ncycleanimation(6, 0.5, 50)

The blue dotted line shows the uniform distribution (1/n), which is the stationary distribution for this Markov chain. By ergodicity, the proportion of time spent at each state will approach the uniform distribution as the walk progresses.